# Wildfire and call simulations

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import numpy as np

import folium
import folium.plugins
from process.folium_patch import HeatMapWithTimeAdditional

from process.simulation_calls import SimulationCalls
from process.simulation_fire import SimulationFire

In [3]:
# GEOGRAPHY
# Coordinates
map_center_coordinates = (39.759132, -121.619311) # Paradise # (40.750536, -122.548867)
map_radius = 1e-4

# Grid parameters
top_left = (39.903571, -121.867089) # Paradise 
bottom_right = (39.625599, -121.345045) # Paradise
grid_width = 0.001

# Epicenters for simulation
random_epicenters = False
human_epicenters = {'UC Berkeley': (37.873248, -122.259546),
                    #"People's Park": (37.867260, -122.257680),
                    'Ohlone Park': (37.872816, -122.279073),
                    'Presidio': (37.796013, -122.458060),
                    'Golden Gate Park': (37.769054, -122.466374),
                    'Forest Knolls': (37.760245, -122.458546),
                    'Bernal Heights Park': (37.744357, -122.415427),
                    'Nob Hill': (37.792127, -122.412512),
                    'Oakland Park': (37.831823, -122.241672)
                   }

background_epicenters = {'San Francisco': (37.762666, -122.445119),
                         'Oakland': (37.812096, -122.235515),
                         'Berkeley': (37.878219, -122.267008),                        
                        }

# Fire propagation
transfer_matrix = np.array([[0,1,0],[0,-0.8,1],[0,0,0]])

# Simulation parameters
n_steps = 50

In [8]:
calls_sim = SimulationCalls(map_center=map_center_coordinates,
                            map_radius=map_radius,
                            n_steps=50,
                            call_centers=human_epicenters,
                            background_call_centers=background_epicenters,        
                           )
calls_df=calls_sim.run()

In [9]:
fire_sim = SimulationFire(map_top_left=top_left,
               map_bottom_right=bottom_right,
               n_steps=50,
               grid_width=0.001,
               transfer_matrix=transfer_matrix,
              )
fire_df=fire_sim.run()

In [14]:
def show_map(static=True):
    """
    Plot realtime fire map using folium.
    Input:
        static: bool. Default: True. Set to False to plot dynamic time map.
    Return:
        None
    """
    
    er_map = folium.Map(location=map_center_coordinates,
                          #tiles="CartoDB dark_matter",
                          width = 1*400,
                          height = 1*300,
                          zoom_start=10,
                         )
   
    # Static map
    if static:
        fire_heatmap = folium.plugins.HeatMap(fire_df[['latitude', 'longitude']].values, radius=10)
        fire_heatmap.layer_name = 'Fire'
        fire_heatmap.add_to(er_map)
        
        
        call_heatmap = folium.plugins.HeatMap(fire_df[['latitude', 'longitude']].values, radius=10)
        call_heatmap.layer_name = 'Calls'
        call_heatmap.add_to(er_map)
        

    # Dynamic map
    else:
        fire_data = [[[row['latitude'],row['longitude']] for _, row in fire_df[fire_df['time']==i].iterrows()] for i in range(0, n_steps)]
        #start_date = datetime.datetime.now()
        #start_date = simulation_start
        #names = [(start_date+ i*datetime.timedelta(seconds=simulation_time_delta)).strftime('%D-%H-%M-%S') for i in range(len(heat_data))]
        
        fire_heatmap = folium.plugins.HeatMapWithTime(fire_data, 
                                                 #index=list(call_times), 
                                                 radius = 10,
                                                 auto_play=True,
                                                 #index=names
                                                 )
        fire_heatmap.add_to(er_map)
        fire_heatmap.layer_name = 'Fire'
        
        
        call_data = [[[row['latitude'],row['longitude']] for _, row in calls_df[calls_df['time']==i].iterrows()] for i in range(0, n_steps)]
        call_heatmap = HeatMapWithTimeAdditional(call_data)
        call_heatmap.layer_name = 'Calls'
        call_heatmap.add_to(er_map)
        
    folium.LayerControl().add_to(er_map)

    return er_map

show_map(static=False)    